## **Without Standardize Method**


In [1]:
import random

In [ ]:
# lets creat a llm component


class MyLLM:
    def __init__(self):
        print("LLM created")

    # our llm will get a prompt and based on the prompt our llm will response
    def predict(self, prompt):
        # create random list and for every prompt

        response = [
            "Delhi is the capital of India",
            "IPL is a cricket league",
            "AI stands for Artificial Intelligence",
        ]

        # randomly choose any response
        return {"response": random.choice(response)}


In [3]:
llm = MyLLM()

LLM created


In [ ]:
llm.predict("what is ai")

{'response': 'AI stands for Artificial Intelligence'}

In [ ]:
class myPromptTemplate:
    def __init__(self, template, input_variable):
        self.template = template
        self.input_variable = input_variable

    # prompts has a format method
    def myformat(self, input_dict):
        # we are calling this function within itself with template to have this input dict and using start start for no limit
        return self.template.format(**input_dict)


In [ ]:
template = myPromptTemplate(
    template="Write a song about {topic}", input_variable=["topic"]
)

In [ ]:
template.myformat({"topic": "India"})

'Write a song about India'

In [ ]:
# we can pass multiple input variables
template = myPromptTemplate(
    template="Write a {length} song about {topic}", input_variable=["topic", "length"]
)

In [ ]:
template.myformat({"topic": "India", "length": "short"})

'Write a short song about India'

Now we need to make a llm application based on both these class, first create llm and make prompt and make


In [ ]:
# Sample Components connecting

# llm create
llm = MyLLM()

prompt = myPromptTemplate(
    template="This is a sample for my Prompt Template for {type} llm",
    input_variable=["type"],
)

prompt = prompt.myformat({"type": "custom"})

llm.predict(prompt)

LLM created


{'response': 'IPL is a cricket league'}

Now we need to Chain those components


In [ ]:
class MyLLMChain:
    def __init__(self, llm, prompt):
        self.llm = llm
        self.prompt = prompt

    def my_chain_run(self, input_dict):
        final_prompt = self.prompt.myformat(input_dict)
        result = self.llm.predict(final_prompt)
        # extract string from the response
        return result["response"]

In [ ]:
# prompt
prompt = myPromptTemplate(
    template="This is a sample for my Prompt Template for {type} llm",
    input_variable=["type"],
)

In [29]:
llm = MyLLM()

LLM created


In [ ]:
chain = MyLLMChain(llm, prompt)

In [ ]:
chain.my_chain_run({"type": "custom"})

'AI stands for Artificial Intelligence'

This is how the Team of Langchain thought to create chains We can see we only need to pass the input at one time and by only calling one function we get the output for both


## **With Standardize Method**


We need to standardize both the class to make flexible chains

1.  Make components Standardize
2.  Chain those components

---

How we are going to do so.

- Convert all components into `runnable`
- All components have same methods --> this is Solid Application of Abstraction so that each component have same methods which we can make sure


In [34]:
from abc import ABC, abstractmethod

In [ ]:
# Create runnable as Abstract class
class Runnable(ABC):
    @abstractmethod
    def invoke(input_data):
        pass
        # This will make sure that each class that inherits from Runnable has invoke method

In [ ]:
class MyLLM(Runnable):
    def __init__(self):
        print("LLM created")

    def predict(self, prompt):
        response = [
            "Delhi is the capital of India",
            "IPL is a cricket league",
            "AI stands for Artificial Intelligence",
        ]

        return {"response": random.choice(response)}


In [37]:
llm = MyLLM()

TypeError: Can't instantiate abstract class MyLLM without an implementation for abstract method 'invoke'

In [ ]:
class MyLLM(Runnable):
    def __init__(self):
        print("LLM created")

    def predict(self, prompt):
        response = [
            "Delhi is the capital of India",
            "IPL is a cricket league",
            "AI stands for Artificial Intelligence",
        ]

        print("This method is going to be depriciated in future with the invoke method")
        return {"response": random.choice(response)}

    # This will work same but with its own implementation
    def invoke(self, prompt):
        response = [
            "Delhi is the capital of India",
            "IPL is a cricket league",
            "AI stands for Artificial Intelligence",
        ]

        return {"response": random.choice(response)}


In [ ]:
class myPromptTemplate(Runnable):
    def __init__(self, template, input_variable):
        self.template = template
        self.input_variable = input_variable

    def myformat(self, input_dict):
        print("This method is going to be depriciated in future with invoke() method")
        return self.template.format(**input_dict)

    def invoke(self, input_dict):
        return self.template.format(**input_dict)


In [ ]:
class RunnableConnector(Runnable):
    def __init__(self, runnable_list):
        self.runnable_list = runnable_list

    def invoke(self, input_data):
        for runnable in self.runnable_list:
            # runnable.invoke(input_data)
            # we will store the output as input for next step
            input_data = runnable.invoke(input_data)

        # after loop ends
        return input_data

In [45]:
"""Now we can run chains"""

'Now we can run chains'

In [ ]:
template = myPromptTemplate(
    template="Write a {length} song about {topic}", input_variable=["topic", "length"]
)

In [47]:
llm = MyLLM()

LLM created


In [49]:
chain = RunnableConnector([template, llm])

In [ ]:
chain.invoke({"topic": "India", "length": "Custom"})

{'response': 'AI stands for Artificial Intelligence'}

**Now main feature of this is we can pass any component through this chain and it will work similar**


In [ ]:
class MyStringOutParser(Runnable):
    def __init__(self) -> None:
        pass

    def invoke(self, input_str):
        return input_str["response"]

In [52]:
parser = MyStringOutParser()

In [53]:
chain = RunnableConnector([template, llm, parser])

In [ ]:
chain.invoke({"topic": "India", "length": "Custom"})

'IPL is a cricket league'

Now here we have seen that we have connected parser also with so ease. Just need to pass the parser


## **Now Connect Two Chains**

1. 1st chain will be of making jokes
2. 2nd chain will be of explaining jokes


In [ ]:
template1 = myPromptTemplate(
    template="Write a joke about {topic}", input_variable=["topic"]
)

In [ ]:
template2 = myPromptTemplate(
    template="Explain the following joke {response}", input_variable=["response"]
)

In [57]:
llm = MyLLM()

LLM created


In [58]:
parser = MyStringOutParser()

In [60]:
# Chain 1
chain1 = RunnableConnector([template1, llm])

In [ ]:
chain1.invoke({"topic": "Cricket"})

{'response': 'Delhi is the capital of India'}

In [63]:
# Chain 2
chain2 = RunnableConnector([template2, llm, parser])

In [ ]:
chain2.invoke({"response": "Cricket"})

'Delhi is the capital of India'

In [ ]:
# Now here is the magic where we can connect those two chains

final_chain = RunnableConnector([chain1, chain2])

final_chain.invoke({"topic": "AI"})

'Delhi is the capital of India'